In [1]:
import torch
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import SNAPDataset

C:\Users\Abhijit\Documents\GitHub\cpsc490\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Homogeneous, Social Network

In [3]:
transform = T.Compose(
    [
        T.ToDevice(device),
        T.RemoveIsolatedNodes(),
        T.RandomLinkSplit(
            num_val=0.05,
            num_test=0.1,
            is_undirected=True,
            add_negative_train_samples=False,
        ),
    ]
)

In [4]:
dataset = SNAPDataset(
    root="./data/SNAPDataset", name="ego-facebook", transform=transform
)

train_data = next(iter(DataLoader([x[0] for x in dataset], batch_size=10)))
val_data = next(iter(DataLoader([x[1] for x in dataset], batch_size=10)))
test_data = next(iter(DataLoader([x[2] for x in dataset], batch_size=10)))

In [5]:
print(test_data)

DataBatch(x=[4167, 1406], edge_index=[2, 160324], circle=[4233], circle_batch=[4233], edge_label=[17800], edge_label_index=[2, 17800], batch=[4167], ptr=[11])


In [6]:
train_data = train_data.to_heterogeneous(node_type_names=["person"], edge_type_names=[("person", "to", "person")])
train_data[("person", "to", "person")]["edge_label"] = train_data["person"].edge_label
train_data[("person", "to", "person")]["edge_label_index"] = train_data["person"].edge_label_index
del train_data["person"].edge_label
del train_data["person"].edge_label_index
del train_data["person"].circle
del train_data["person"].circle_batch

val_data = val_data.to_heterogeneous(node_type_names=["person"], edge_type_names=[("person", "to", "person")])
val_data[("person", "to", "person")]["edge_label"] = val_data["person"].edge_label
val_data[("person", "to", "person")]["edge_label_index"] = val_data["person"].edge_label_index
del val_data["person"].edge_label
del val_data["person"].edge_label_index
del val_data["person"].circle
del val_data["person"].circle_batch

test_data = test_data.to_heterogeneous(node_type_names=["person"], edge_type_names=[("person", "to", "person")])
test_data[("person", "to", "person")]["edge_label"] = test_data["person"].edge_label
test_data[("person", "to", "person")]["edge_label_index"] = test_data["person"].edge_label_index
del test_data["person"].edge_label
del test_data["person"].edge_label_index
del test_data["person"].circle
del test_data["person"].circle_batch

In [7]:
print(train_data)
print(val_data)
print(test_data)

HeteroData(
  person={
    x=[4167, 1406],
    batch=[4167],
    ptr=[11]
  },
  (person, to, person)={
    edge_index=[2, 151430],
    edge_label=[75715],
    edge_label_index=[2, 75715]
  }
)
HeteroData(
  person={
    x=[4167, 1406],
    batch=[4167],
    ptr=[11]
  },
  (person, to, person)={
    edge_index=[2, 151430],
    edge_label=[8894],
    edge_label_index=[2, 8894]
  }
)
HeteroData(
  person={
    x=[4167, 1406],
    batch=[4167],
    ptr=[11]
  },
  (person, to, person)={
    edge_index=[2, 160324],
    edge_label=[17800],
    edge_label_index=[2, 17800]
  }
)


## Heterogeneous, Recommender System

### DBLP

In [8]:
from torch_geometric.datasets import DBLP

In [9]:
transform = T.Compose([
    T.ToDevice(device),
    T.RemoveIsolatedNodes(),
    T.RandomLinkSplit(
        num_val=0.05, 
        num_test=0.1, 
        is_undirected=True, 
        add_negative_train_samples=False,
        edge_types=[("paper", "to", "author")]
    ),
    T.ToUndirected(),
])

In [10]:
dataset = DBLP(root="../data/DBLP", transform=transform)

train_data, val_data, test_data = dataset[0]

for data in train_data, val_data, test_data:
    del data["term"]
    del data[("paper", "to", "term")]
    del data[("term", "to", "paper")]
    del data[("author", "to", "paper")]
    del data[("conference", "to", "paper")]
    
    del data[("paper", "rev_to", "author")]
    del data[("term", "rev_to", "paper")]
    del data[("paper", "rev_to", "term")]
    del data[("paper", "rev_to", "conference")]
    
    del data["author"].train_mask
    del data["author"].val_mask
    del data["author"].test_mask
    del data["author"].y
    
    data["conference"].x = torch.ones((20, 1))
    del data["conference"].num_nodes

In [11]:
train_data

HeteroData(
  author={ x=[4057, 334] },
  paper={ x=[14328, 4231] },
  conference={ x=[20, 1] },
  (paper, to, author)={
    edge_index=[2, 16699],
    edge_label=[16699],
    edge_label_index=[2, 16699]
  },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (author, rev_to, paper)={
    edge_index=[2, 16699],
    edge_label=[16699]
  },
  (conference, rev_to, paper)={ edge_index=[2, 14328] }
)

In [12]:
val_data

HeteroData(
  author={ x=[4057, 334] },
  paper={ x=[14328, 4231] },
  conference={ x=[20, 1] },
  (paper, to, author)={
    edge_index=[2, 16699],
    edge_label=[1964],
    edge_label_index=[2, 1964]
  },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (author, rev_to, paper)={ edge_index=[2, 16699] },
  (conference, rev_to, paper)={ edge_index=[2, 14328] }
)

In [13]:
test_data

HeteroData(
  author={ x=[4057, 334] },
  paper={ x=[14328, 4231] },
  conference={ x=[20, 1] },
  (paper, to, author)={
    edge_index=[2, 17681],
    edge_label=[3928],
    edge_label_index=[2, 3928]
  },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (author, rev_to, paper)={ edge_index=[2, 17681] },
  (conference, rev_to, paper)={ edge_index=[2, 14328] }
)

### IMDB

In [14]:
from torch_geometric.datasets import IMDB

In [15]:
transform = T.Compose([
    T.ToDevice(device),
    T.RemoveIsolatedNodes(),
    T.RandomLinkSplit(
        num_val=0.05, 
        num_test=0.1, 
        is_undirected=True, 
        add_negative_train_samples=False,
        edge_types=[("movie", "to", "actor")]
    ),
    T.ToUndirected(),
])

In [16]:
dataset = IMDB(root="../data/IMDB", transform=transform)

train_data, val_data, test_data = dataset[0]

for data in train_data, val_data, test_data:
    del data[("director", "to", "movie")]
    del data[("actor", "to", "movie")]
    
    del data[("movie", "rev_to", "director")]
    del data[("movie", "rev_to", "actor")]
    
    del data["movie"].train_mask
    del data["movie"].val_mask
    del data["movie"].test_mask
    del data["movie"].y

In [17]:
train_data

HeteroData(
  movie={ x=[4278, 3066] },
  director={ x=[2081, 3066] },
  actor={ x=[5257, 3066] },
  (movie, to, director)={ edge_index=[2, 4278] },
  (movie, to, actor)={
    edge_index=[2, 10905],
    edge_label=[10905],
    edge_label_index=[2, 10905]
  },
  (director, rev_to, movie)={ edge_index=[2, 4278] },
  (actor, rev_to, movie)={
    edge_index=[2, 10905],
    edge_label=[10905]
  }
)

In [18]:
val_data

HeteroData(
  movie={ x=[4278, 3066] },
  director={ x=[2081, 3066] },
  actor={ x=[5257, 3066] },
  (movie, to, director)={ edge_index=[2, 4278] },
  (movie, to, actor)={
    edge_index=[2, 10905],
    edge_label=[1282],
    edge_label_index=[2, 1282]
  },
  (director, rev_to, movie)={ edge_index=[2, 4278] },
  (actor, rev_to, movie)={ edge_index=[2, 10905] }
)

In [19]:
test_data

HeteroData(
  movie={ x=[4278, 3066] },
  director={ x=[2081, 3066] },
  actor={ x=[5257, 3066] },
  (movie, to, director)={ edge_index=[2, 4278] },
  (movie, to, actor)={
    edge_index=[2, 11546],
    edge_label=[2564],
    edge_label_index=[2, 2564]
  },
  (director, rev_to, movie)={ edge_index=[2, 4278] },
  (actor, rev_to, movie)={ edge_index=[2, 11546] }
)